In [1]:
from helper import Helper
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from time import time

In [2]:
h = Helper()

train_it, validation_it, test_it = h.construct_data_generator_w_validation(batch_size=10, rescale=1./255.)

Found 22050 images belonging to 3 classes.
Found 3150 images belonging to 3 classes.
Found 6300 images belonging to 3 classes.


In [3]:
model = ResNet50(include_top=True, weights=None, input_shape=(224,224,3), classes=3)
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [4]:
# x = resnet.output
# x = GlobalAveragePooling2D()(x)
# x = Dropout(0.5)(x)
# pred = Dense(3, activation='softmax')(x)
# model = Model(inputs=resnet.input, outputs=pred, name='cnn_topomap_classification_model_resnet')
# model.summary()

In [5]:
opt = Adam(learning_rate=0.0001)
loss = 'categorical_crossentropy'

model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [6]:
tblog_path = h.logs_directory.format(time())
tb_log = TensorBoard(log_dir = tblog_path, profile_batch=0, write_images=True)

ch = ModelCheckpoint('models/resnet/cnn_topomap_classification_model_resnet.h5', monitor='val_accuracy', 
                     mode='max', save_best_only=True)

In [7]:
r = model.fit_generator(train_it, epochs=100, callbacks=[ch, tb_log], validation_data=validation_it)

Epoch 1/100
2205/2205 [==============================] - 457s 207ms/step - loss: 1.1647 - accuracy: 0.3730 - val_loss: 1.4555 - val_accuracy: 0.3530
Epoch 2/100
2205/2205 [==============================] - 461s 209ms/step - loss: 1.0360 - accuracy: 0.4776 - val_loss: 1.4973 - val_accuracy: 0.3702
Epoch 3/100
2205/2205 [==============================] - 460s 208ms/step - loss: 0.8717 - accuracy: 0.5904 - val_loss: 0.9424 - val_accuracy: 0.5571
Epoch 4/100
2205/2205 [==============================] - 460s 208ms/step - loss: 0.7556 - accuracy: 0.6589 - val_loss: 0.8726 - val_accuracy: 0.5990
Epoch 5/100
2205/2205 [==============================] - 457s 207ms/step - loss: 0.6503 - accuracy: 0.7195 - val_loss: 1.2663 - val_accuracy: 0.4924
Epoch 6/100
2205/2205 [==============================] - 459s 208ms/step - loss: 0.5759 - accuracy: 0.7563 - val_loss: 0.8122 - val_accuracy: 0.6343
Epoch 7/100
2205/2205 [==============================] - 457s 207ms/step - loss: 0.5054 - accuracy: 0.7880

Epoch 56/100
2205/2205 [==============================] - 459s 208ms/step - loss: 0.0746 - accuracy: 0.9721 - val_loss: 0.2898 - val_accuracy: 0.9019
Epoch 57/100
2205/2205 [==============================] - 458s 208ms/step - loss: 0.0764 - accuracy: 0.9722 - val_loss: 0.3577 - val_accuracy: 0.8832
Epoch 58/100
2205/2205 [==============================] - 457s 207ms/step - loss: 0.0721 - accuracy: 0.9745 - val_loss: 0.4666 - val_accuracy: 0.8565
Epoch 59/100
2205/2205 [==============================] - 459s 208ms/step - loss: 0.0732 - accuracy: 0.9736 - val_loss: 0.2477 - val_accuracy: 0.9229
Epoch 60/100
2205/2205 [==============================] - 458s 208ms/step - loss: 0.0732 - accuracy: 0.9725 - val_loss: 0.2820 - val_accuracy: 0.9146
Epoch 61/100
2205/2205 [==============================] - 458s 208ms/step - loss: 0.0685 - accuracy: 0.9750 - val_loss: 0.2852 - val_accuracy: 0.9057
Epoch 62/100
2205/2205 [==============================] - 458s 208ms/step - loss: 0.0665 - accuracy:

In [8]:
evaluate = model.evaluate_generator(test_it)

In [9]:
y_prob = model.predict_generator(test_it)

In [10]:
h.save(model, r, evaluate, y_prob, 'cnn_topomap_classification_model_resnet')

model saved, path: models/cnn_topomap_classification_model_resnet.h5


In [11]:
model_ch = h.load('models/resnet/cnn_topomap_classification_model_resnet.h5')

In [12]:
e_ch = model_ch.evaluate_generator(test_it)

In [13]:
e_ch

[0.24716005615818365, 0.9285714]

In [14]:
plt.figure()

plt.title('loss curve for cnn_topomap_classification_model_resnet')

plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')

plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

plt.savefig('results/loss curve for cnn_topomap_classification_model_resnet.png')

NameError: name 'plt' is not defined

In [ ]:
plt.figure()

plt.title('accuracy curve for cnn_topomap_classification_model_resnet')

plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label='val_acc')

plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

plt.savefig('results/accuracy curve for cnn_topomap_classification_model_resnet.png')